<a href="https://colab.research.google.com/github/detektor777/colab_list_image/blob/main/Real_ESRGAN_photo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title ##**Install** { display-mode: "form" }
!nvidia-smi
!git clone https://github.com/xinntao/Real-ESRGAN.git
%cd Real-ESRGAN
# Set up the environment
!pip install basicsr
!pip install facexlib
!pip install gfpgan
!pip install -r requirements.txt
!python setup.py develop

import shutil
from tqdm import tqdm
import os
import shutil, sys
import re
import io
import IPython.display
import numpy as np
import PIL.Image
from google.colab import files
import shutil

# URL новой модели
new_model_path = 'https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.4.pth'

# Открываем файл локально и читаем его содержимое
filename = '/content/Real-ESRGAN/inference_realesrgan.py'
with open(filename, 'r') as f:
    script_content = f.read()

# Изменяем путь к модели в строке скрипта
new_script_content = re.sub(r"(model_path\s*=\s*[\"\']).*?([\"\'])", rf"\g<1>{new_model_path}\g<2>", script_content)

# Записываем измененный скрипт в файл
with open(filename, 'w') as f:
    f.write(new_script_content)

!sed -i 's/from torchvision.transforms.functional_tensor import rgb_to_grayscale/from torchvision.transforms.functional import rgb_to_grayscale/' /usr/local/lib/python3.10/dist-packages/basicsr/data/degradations.py


In [ ]:
#@title ##**Upload images** { display-mode: "form" }
%cd /content/Real-ESRGAN
#input_folder = "test_images/old"
#output_folder = "output"

upload_folder = "/content/Real-ESRGAN/upload"
result_folder = "/content/Real-ESRGAN/results"

if os.path.isdir(upload_folder):
    shutil.rmtree(upload_folder)
os.makedirs(upload_folder)

basepath = os.getcwd()
uploaded = files.upload()
for filename in uploaded.keys():
    shutil.move(os.path.join(basepath, filename), os.path.join(upload_folder, filename))

In [ ]:
#@title ##**Run** { display-mode: "form" }

if os.path.isdir(result_folder):
    shutil.rmtree(result_folder)
os.makedirs(result_folder)

model_name = "RealESRGAN_x2plus" #@param ["RealESRGAN_x2plus","RealESRGAN_x4plus","RealESRNet_x4plus","realesr-general-x4v3","RealESRGAN_x4plus_anime_6B","realesr-animevideov3"]
scale = "2" #@param ["1","2","3","4"]
face_enhance = "Yes" #@param ["Yes","No"]

command = "python inference_realesrgan.py -n {} -i upload --outscale {}".format(model_name, scale)
if face_enhance == "Yes":
    command += " --face_enhance"

print(command)
os.system(command)
#!python inference_realesrgan.py -n RealESRGAN_x2plus -i upload --outscale 2

In [ ]:
#@title ##**Visualize** { display-mode: "form" }
filenames_upload = os.listdir(upload_folder)
filenames_upload.sort()

filenames_upload_output = os.listdir(result_folder)
filenames_upload_output.sort()

for filename, filename_output in zip(filenames_upload, filenames_upload_output):
    image_original = PIL.Image.open(os.path.join(upload_folder, filename))
    image_restore = PIL.Image.open(os.path.join(result_folder, filename_output))

    # Resize images to have a maximum width of 600 pixels
    max_width = 500
    width_original, height_original = image_original.size
    width_restore, height_restore = image_restore.size
    if width_original > max_width:
        new_height = int(height_original * max_width / width_original)
        image_original = image_original.resize((max_width, new_height))
    if width_restore > max_width:
        new_height = int(height_restore * max_width / width_restore)
        image_restore = image_restore.resize((max_width, new_height))

    # Combine and display images
    display(PIL.Image.fromarray(np.hstack((np.array(image_original), np.array(image_restore)))))
    print("")

In [ ]:
#@title ##**Download results** { display-mode: "form" }
#output_folder = os.path.join(upload_output_path, "final_output")
files_in_folder = os.listdir(result_folder)
zip_file = "download.zip"

if len(files_in_folder) == 1:
    file_to_download = os.path.join(result_folder, files_in_folder[0])
    files.download(file_to_download)
else:
    if os.path.exists(os.path.join(result_folder, zip_file)):
      os.remove(os.path.join(result_folder, zip_file))
    os.system(f"cd {result_folder} && zip -r -j {zip_file} * && cd ..")
    files.download(os.path.join(result_folder, zip_file))

In [ ]:
#@title ##**Download results to google drive (optional)** { display-mode: "form" }
from google.colab import drive
drive.mount('/content/drive')

!cp "/content/Real-ESRGAN/results/download.zip" "/content/drive/MyDrive/"
